In [5]:
import lamini 
import logging
import sqlite3
import pandas as pd
from util.get_schema import get_schema
from util.make_llama_3_prompt import make_llama_3_prompt
from util.setup_logging import setup_logging

logger = logging.getLogger(__name__)
engine = sqlite3.connect("./nba_roster.db")
setup_logging()

In [14]:
lamini.api_key = "e650b0aaad19a8ac43752657c882cc3405398636449d41b947d80c399f1d0407"
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [15]:
# Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt. 
# You can find the documentation on this [model card](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-instruct).
def make_llama_3_prompt(user, system=""):
    system_prompt = ""
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [16]:
def get_schema():
    return """\
0|Team|TEXT 
1|NAME|TEXT  
2|Jersey|TEXT 
3|POS|TEXT
4|AGE|INT 
5|HT|TEXT 
6|WT|TEXT 
7|COLLEGE|TEXT 
8|SALARY|TEXT eg. 
"""

In [17]:
user = """Who is the highest paid NBA player?"""

In [18]:
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

In [19]:
print(system)

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|Team|TEXT 
1|NAME|TEXT  
2|Jersey|TEXT 
3|POS|TEXT
4|AGE|INT 
5|HT|TEXT 
6|WT|TEXT 
7|COLLEGE|TEXT 
8|SALARY|TEXT eg. 


Write a sqlite query to answer the following question. Follow instructions exactly


In [20]:
prompt = make_llama_3_prompt(user, system)

In [21]:
print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
ORDER BY SALARY DESC
LIMIT 1;
```

This query will return the name and salary of the highest paid NBA player. The `ORDER BY SALARY DESC` clause sorts the players by their salary in descending order (highest to lowest), and the `LIMIT 1` clause ensures that we only get the top result.


In [22]:
def get_updated_schema():
    return """\
0|Team|TEXT eg. "Toronto Raptors"
1|NAME|TEXT eg. "Otto Porter Jr."
2|Jersey|TEXT eg. "0" and when null has a value "NA"
3|POS|TEXT eg. "PF"
4|AGE|INT eg. "22" in years
5|HT|TEXT eg. `6' 7"` or `6' 10"`
6|WT|TEXT eg. "232 lbs" 
7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"
"""

In [23]:
prompt = make_llama_3_prompt(user, system)

In [24]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|Team|TEXT 
1|NAME|TEXT  
2|Jersey|TEXT 
3|POS|TEXT
4|AGE|INT 
5|HT|TEXT 
6|WT|TEXT 
7|COLLEGE|TEXT 
8|SALARY|TEXT eg. 


Write a sqlite query to answer the following question. Follow instructions exactly<|eot_id|><|start_header_id|>user<|end_header_id|>

Who is the highest paid NBA player?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [25]:
print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
ORDER BY SALARY DESC
LIMIT 1;
```

This query will return the name and salary of the highest paid NBA player. The `ORDER BY SALARY DESC` clause sorts the players by their salary in descending order (highest to lowest), and the `LIMIT 1` clause ensures that we only get the top result.


In [26]:
result = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)

In [27]:
result

{'sqlite_query': 'SELECT NAME, SALARY FROM nba_roster ORDER BY SALARY DESC LIMIT 1'}

In [28]:
df = pd.read_sql(result['sqlite_query'], con=engine)

In [29]:
df

,NAME,SALARY
0,Trent Forrest,--


In [30]:
query="""SELECT salary, name 
FROM nba_roster 
WHERE salary != '--' 
ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC 
LIMIT 1;"""
df = pd.read_sql(query, con=engine)
print(df)

        SALARY           NAME
0  $51,915,615  Stephen Curry
